This notebook includes markdown cells (such as this one), which provide useful information and narration of what is happening, and code cells (such as the next one), which actually do things.  To execute a code cells, click in the cell and then hit Shift+Enter.  Alternatively, click within a cell and hit the Run button (play symbol).  When a cell completes executing, a number will appear (or update) on the left-hand side of the cell. You can also execute some or all of the cells via options in the 'Runtime' menu, above. 

Before we do anything else, we need to install some required pacakges, download some data, and set up our Python environment:

In [ ]:
# this can take up to a minute or longer
!git clone --depth 1 https://github.com/dsavransky/YieldModelingWorkshopTutorial

In [ ]:
!pip install EXOSIMS ipympl

In [ ]:
import os
import shutil
edir = os.path.join(os.environ['HOME'],'.EXOSIMS')
os.makedirs(edir, exist_ok=True)
shutil.copytree("YieldModelingWorkshopTutorial/cache", os.path.join(edir, "cache"),dirs_exist_ok=True)
shutil.copytree("YieldModelingWorkshopTutorial/downloads",os.path.join(edir, "downloads"),dirs_exist_ok=True)

In [ ]:
# import all of the packages we're going to use
import matplotlib.pyplot as plt
import numpy as np
import copy
import EXOSIMS.MissionSim
import astropy.units as u
import scipy
from matplotlib import ticker
import matplotlib.colors
import warnings

In [ ]:
# set up plotting
from google.colab import output
output.enable_custom_widget_manager()

%matplotlib widget
plt.rcParams.update({"figure.max_open_warning": 0})

We recommend executing the code cells below one by one and interacting with the outputs to get the most out of this tutorial. You can also skip to the start of any numbered section and start execution from there, but all cells within a section are intended to be executed in order.

# 1. Building and Interacting with Simulation Objects

In order to create a mission simulation, we need an input specification.  Let's define one with all default values, except for a real input star catalog:

In [ ]:
specs0 = {
    "modules": {
        "PlanetPopulation": " ",
        "StarCatalog": "HWOMissionStars",
        "OpticalSystem": " ",
        "ZodiacalLight": " ",
        "BackgroundSources": " ",
        "PlanetPhysicalModel": " ",
        "Observatory": " ",
        "TimeKeeping": " ",
        "PostProcessing": " ",
        "Completeness": " ",
        "TargetList": " ",
        "SimulatedUniverse": " ",
        "SurveySimulation": " ",
        "SurveyEnsemble": " ",
    }
}

We now create a `MissionSim` object, which will automatically build and pre-compute all the quantities we need to simulate a mission. Note that for this tutorial, pre-computed values have been automatically loaded to speed up execution time.  Running the same commands with different inputs can result in *much* longer execution times.

In [ ]:
sim0 = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs0))

Note that a *lot* of defaults have been set. Let's take a look at some of the default values being used, starting with which modules were loaded:

In [ ]:
sim0.modules

These modules make up the `EXOSIMS` framework and are all available as attributes of the top-level `MissionSim` object (the one we just created as `sim`).  Regardless of which specific implementations get loaded, the attribute names referring to them are always the same. Certain modules are also available as attributes of others, based on `EXOSIMS` instantiation tree (see here for details: https://exosims.readthedocs.io/en/latest/intro.html#framework). As an example, we can show that the `PlanetPopulation` attribute of `MissionSim` is the same as the `PlanetPopulation` attribute of the `TargetList`. 

In [ ]:
sim0.PlanetPopulation is sim0.TargetList.PlanetPopulation

Next, we can explore specific parameters that are user-settable (but in this case got set to default values). We start with the `OpticalSystem` (https://exosims.readthedocs.io/en/latest/opticalsystem.html):

In [ ]:
print(f"Pupil Diameter: {sim0.OpticalSystem.pupilDiam}")
print(f"Single-observation integration time limit: {sim0.OpticalSystem.intCutoff}")
print(
    f"Observing mode central wavelength: {sim0.OpticalSystem.observingModes[0]['lam']}"
)
print(
    f"Observing mode IWA/OWA: {sim0.OpticalSystem.observingModes[0]['IWA']}/ "
    f"{sim0.OpticalSystem.observingModes[0]['OWA']}"
)

We can also see what kind of contrast and throughput we're expecting.  These are both functions of wavelength and angular separation (see https://exosims.readthedocs.io/en/latest/opticalsystem.html#required-prototype-starlight-suppression-system-parameters for details). We'll check at the central wavelength and inner working angle:

In [ ]:
print(
    "Starlight Suppression system contrast @0.1 arcsec: "
    f"{sim0.OpticalSystem.starlightSuppressionSystems[0]['core_contrast'](500*u.nm, 0.1*u.arcsec)[0] :.2e}"
)
print(
    "Starlight Suppression system core throughput @0.1 arcsec: "
    f"{sim0.OpticalSystem.starlightSuppressionSystems[0]['core_thruput'](500*u.nm, 0.1*u.arcsec)[0]}"
)

We can also get some information about our science instrument (https://exosims.readthedocs.io/en/latest/opticalsystem.html#science-instruments):

In [ ]:
print(
    "Science Instrument QE @500 nm: "
    f"{sim0.OpticalSystem.scienceInstruments[0]['QE'](500*u.nm)[0]}"
)
print(
    f"Science Instrument pixel scale: {sim0.OpticalSystem.scienceInstruments[0]['pixelScale']}"
)
print(
    f"Science Instrument dark current: {sim0.OpticalSystem.scienceInstruments[0]['idark']}"
)
print(
    f"Science Instrument read noise (e): {sim0.OpticalSystem.scienceInstruments[0]['sread']}"
)

Let's also take a look at our target list (https://exosims.readthedocs.io/en/latest/targetlist.html):

In [ ]:
print(f"Number of targets: {sim0.TargetList.nStars}")
print(f"Min/Max Target V mag: {sim0.TargetList.Vmag.min()}/{sim0.TargetList.Vmag.max()}")
print(
    f"Min/Max Target distance: {sim0.TargetList.dist.min()}/{sim0.TargetList.dist.max()}"
)

If you read the HWO Mission star list documentation (https://exoplanetarchive.ipac.caltech.edu/docs/2645_NASA_ExEP_Target_List_HWO_Documentation_2023.pdf), you'll notice that it includes 164 targets.  Why are only 134 included here?  Let's ask the code to explain:

In [ ]:
# we'll create a new input specification and object so we can
# revisit the original one later, if desired.
specs1 = copy.deepcopy(specs0)
specs1["explainFiltering"] = True
sim1 = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs1))

30 of the targets have bright companions within 10 arcsec, and are removed by default.  We can prevent this, if desired:

In [ ]:
specs1["filterBinaries"] = False
sim1 = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs1))

In [ ]:
print(f"Number of targets: {sim1.TargetList.nStars}")
print(
    f"Min/Max Target V mag: {sim1.TargetList.Vmag.min()}/{sim1.TargetList.Vmag.max()}"
)
print(
    f"Min/Max Target distance: {sim1.TargetList.dist.min()}/{sim1.TargetList.dist.max()}"
)

As we now have two different `MissionSim` objects (and two sets of underlying module objects) we can compare them.  Let's see the exact set of stars that were initially filtered out:

In [ ]:
print(set(sim1.TargetList.Name) - set(sim0.TargetList.Name))

# 2. Target Achievable $\Delta$mag Values

Let's create a new `MissionSim` object with one minor modification: by default, EXOSIMS will not compute the saturation completeness or associated $\Delta\textrm{mag}$ values (effectively infinite integration time - for more details see: https://exosims.readthedocs.io/en/latest/concepts.html#completeness-integration-time-and-delta-textrm-mag).  However, we can toggle this on by adding a single boolean to our input specification:

In [ ]:
specs1 = {
    "skipSaturationCalcs": False,
    "filterBinaries": False,
    "modules": {
        "PlanetPopulation": " ",
        "StarCatalog": "HWOMissionStars",
        "OpticalSystem": " ",
        "ZodiacalLight": " ",
        "BackgroundSources": " ",
        "PlanetPhysicalModel": " ",
        "Observatory": " ",
        "TimeKeeping": " ",
        "PostProcessing": " ",
        "Completeness": " ",
        "TargetList": " ",
        "SimulatedUniverse": " ",
        "SurveySimulation": " ",
        "SurveyEnsemble": " ",
    }
}
sim1 = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs1))

We can now see how all of these translate to achievable $\Delta\textrm{mag}$ values for our target list.  We'll look at the maximum integration time value and the saturation value:

In [ ]:
TL1 = sim1.TargetList  # assign TargetList object for easier access
plt.figure()
plt.scatter(TL1.Vmag, TL1.intCutoff_dMag, label="Max. Int Time $\\Delta$mag")
plt.scatter(TL1.Vmag, TL1.saturation_dMag, label="Saturation $\\Delta$mag")
plt.legend()
plt.xlabel("V magnitude")
plt.ylabel("$\\Delta$mag");

That's odd.  I thought we had $10^{-10}$ contrast. Why are we saturating at only a bit above 23 and not closer to 25 $\Delta\textrm{mag}$ (which is $10^{-10}$ on the magnitude scale)? 

The reason is that our integration time model assumes a noise floor set by the residual starlight leaking through the coronagraph. This is attenuated by an assumed post-processing gain. There is also a stability factor that models overall PSF stability. Let's see what that these values default to:

In [ ]:
print(f"Post-Processing gain: {TL1.PostProcessing._outspec['ppFact']}")
print(f"Stability Factor: {TL1.OpticalSystem.stabilityFact}")

Well, that's the problem right there.  We're not attenuating the residual speckle at all.  Let's fix that and see how it changes things.  We'll assume that we can beat down the speckle residual (via some form of post-processing) by a factor of 10 and re-create the same plot:

In [ ]:
specs2 = copy.deepcopy(specs1)
specs2["ppFact"] = 0.1
sim2 = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs2))
TL2 = sim2.TargetList
plt.figure()
plt.scatter(TL2.Vmag, TL2.intCutoff_dMag, label="Max. Int Time $\\Delta$mag")
plt.scatter(TL2.Vmag, TL2.saturation_dMag, label="Saturation $\\Delta$mag")
plt.legend()
plt.xlabel("V magnitude")
plt.ylabel("$\\Delta$mag");

Much better! We are now modeling an optical system that has a reasonable chance of being able to detect Earth-like planets.

# 3. Planet Populations

Now let's take a look at what kind of synthetic planets we're creating.  First we make a new `MissionSim` object with all of the settings we've accumulated so far:

In [ ]:
specs2 = {
    "skipSaturationCalcs": False,
    "filterBinaries": False,
    "ppFact": 0.1,
    "modules": {
        "PlanetPopulation": " ",
        "StarCatalog": "HWOMissionStars",
        "OpticalSystem": " ",
        "ZodiacalLight": " ",
        "BackgroundSources": " ",
        "PlanetPhysicalModel": " ",
        "Observatory": " ",
        "TimeKeeping": " ",
        "PostProcessing": " ",
        "Completeness": " ",
        "TargetList": " ",
        "SimulatedUniverse": " ",
        "SurveySimulation": " ",
        "SurveyEnsemble": " ",
    }
}
sim2 = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs2))

Now we can look at multiple attributes to get a sense of the planet sample we're generating:

In [ ]:
print(f"Assumed occurrence rate: {sim2.PlanetPopulation.eta}")
print(f"Number of stars in target list: {sim2.TargetList.nStars}")
print(f"Number of planets in synthetic universe: {sim2.SimulatedUniverse.nPlans}")

You might notice that the number of planets not exactly equal to $\eta N_\textrm{stars}$ (we can't say for sure, because a different random draw occurs for every user of this sheet, every time it is run, so the number of planets you see in the previous output will be different each time). 

The reason why you won't always get exactly $\eta N_\textrm{stars}$ planets is because we treat $\eta$ as the rate parameter of a Poisson random variable.  

We're also not producing a lot of planets, so for visualization purposes, let's increase the occurrence rate:

In [ ]:
specs3 = copy.deepcopy(specs2)
specs3["eta"] = 3
sim3 = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs3))
print(f"Assumed occurrence rate: {sim3.PlanetPopulation.eta}")
print(f"Number of stars in target list: {sim3.TargetList.nStars}")
print(f"Number of planets in synthetic universe: {sim3.SimulatedUniverse.nPlans}")

Let's take a look at some of the planets' physical and orbital attributes:

In [ ]:
fig, ax = plt.subplots()
pts = ax.scatter(
    sim3.SimulatedUniverse.a,
    sim3.SimulatedUniverse.Mp,
    s=sim3.SimulatedUniverse.Rp,
    c=sim3.SimulatedUniverse.Rp,
)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_ylabel(f"Planet Mass [{sim3.SimulatedUniverse.Mp.unit}]")
ax.set_xlabel(f"Semi-Major Axis [{sim3.SimulatedUniverse.a.unit}]")
plt.colorbar(pts, label=f"Planet Radius [{sim3.SimulatedUniverse.Rp.unit}]");

That seems like a really odd-looking planet population.  Because we haven't selected a specific planet population to model, planets were generated using log-normal distributions for mass and semi-major axis, and the planet radius is decoupled from the planet mass.  This is obviously non-physical (although occasionally useful for various testing purposes) - let's dial in a real planet population. We'll use the population defined by the SAG13 final report (https://exoplanets.nasa.gov/system/presentations/files/67_Belikov_SAG13_ExoPAG16_draft_v4.pdf):

In [ ]:
specs4 = copy.deepcopy(specs3)
specs4["modules"]["PlanetPopulation"] = "SAG13"
specs4["modules"]["SimulatedUniverse"] = "SAG13Universe"
sim4 = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs4))
print(f"Assumed occurrence rate: {sim4.PlanetPopulation.eta}")
print(f"Number of stars in target list: {sim4.TargetList.nStars}")
print(f"Number of planets in synthetic universe: {sim4.SimulatedUniverse.nPlans}")

Note that our previously set $\eta$ input was ignored and overwritten by this particular family of modules. 

In [ ]:
fig, ax = plt.subplots()
pts = ax.scatter(
    sim4.SimulatedUniverse.a,
    sim4.SimulatedUniverse.Mp,
    s=sim4.SimulatedUniverse.Rp,
    c=sim4.SimulatedUniverse.Rp,
)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_ylabel(f"Planet Mass [{sim4.SimulatedUniverse.Mp.unit}]")
ax.set_xlabel(f"Semi-Major Axis [{sim4.SimulatedUniverse.a.unit}]")
plt.colorbar(pts, label=f"Planet Radius [{sim4.SimulatedUniverse.Rp.unit}]");

We are now generating planets with self-consistent masses and radii, and generating significantly more Earth-mass objects than Jovian-mass objects. However, the original SAG13 population is only defined for relatively short-period planets.  We can see the exact range of semi-major axes we're generating:

In [ ]:
print(f"Semi-major axis range: {sim4.PlanetPopulation.arange}")

Let's extrapolate a bit:

In [ ]:
specs5 = copy.deepcopy(specs4)
specs5["arange"] = [0.09084645, 30]
specs5["smaknee"] = 10
sim5 = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs5))
print(f"Assumed occurrence rate: {sim5.PlanetPopulation.eta}")
print(f"Number of stars in target list: {sim5.TargetList.nStars}")
print(f"Number of planets in synthetic universe: {sim5.SimulatedUniverse.nPlans}")

fig, ax = plt.subplots()
pts = ax.scatter(
    sim5.SimulatedUniverse.a,
    sim5.SimulatedUniverse.Mp,
    s=sim5.SimulatedUniverse.Rp,
    c=sim5.SimulatedUniverse.Rp,
)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_ylabel(f"Planet Mass [{sim5.SimulatedUniverse.Mp.unit}]")
ax.set_xlabel(f"Semi-Major Axis [{sim5.SimulatedUniverse.a.unit}]")
plt.colorbar(pts, label=f"Planet Radius [{sim5.SimulatedUniverse.Rp.unit}]");

Note that even though we permitted the semi-major axis range to go all the way to 30 AU, we're not generating planets out there.  That's because this implementation includes an exponential drop-off past the separation set by the ``smaknee`` parameter, which we set to 10 AU.

# 4. Completeness

At this point, we have a (somewhat) reasonable-looking set of synthetic planets, a target list, and a workable optical system.  The last element we're missing is the ability compute completeness.  So far, we have been utilizing the prototype completeness modules, which doesn't actually compute real completeness values (in the interest of execution time). Instead, it sets every completeness value for every target to an arbitrary, identical, value:

In [ ]:
specs5 = {
    "skipSaturationCalcs": False,
    "filterBinaries": False,
    "ppFact": 0.1,
    "eta": 3,
    "arange": [0.09084645, 30],
    "smaknee": 10,
    "modules": {
        "PlanetPopulation": "SAG13",
        "StarCatalog": "HWOMissionStars",
        "OpticalSystem": " ",
        "ZodiacalLight": " ",
        "BackgroundSources": " ",
        "PlanetPhysicalModel": " ",
        "Observatory": " ",
        "TimeKeeping": " ",
        "PostProcessing": " ",
        "Completeness": " ",
        "TargetList": " ",
        "SimulatedUniverse": "SAG13Universe",
        "SurveySimulation": " ",
        "SurveyEnsemble": " ",
    }
}
sim5 = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs5))
sim5.TargetList.int_comp

Let's replace this with a real completeness calculation, and also limit ourselves to a somewhat restricted field of view.  Note that this will execute relatively quickly because we have pre-cached the relevant result for this tutorial.  Changing the planet population will require a re-computation of the joint density function, which can take a while (but then those results will be cached as well). We're going to use a Monte-Carlo based approach to computing Completeness, which is named in honor of Bob Brown, who originally introduced this methodology in his 2005 paper:

In [ ]:
specs6 = copy.deepcopy(specs5)
specs6["modules"]["Completeness"] = "BrownCompleteness"
specs6["FoV"] = 2.0  # arcseconds
sim6 = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs6))

We can now look at what kind of completeness values we can achieve for our target list.  We'll consider the case of integrating to a nominal $\Delta$mag (defaulting to 25) for each target, and the case of integrating for an effectively infinite amount of time:

In [ ]:
TL = sim6.TargetList
plt.figure()
plt.scatter(TL.dist, TL.int_comp, label="Nominal integraton time completeness")
plt.scatter(TL.dist, TL.saturation_comp, label="Saturation completeness")
plt.legend()
plt.xlabel(f"Distance {TL.dist.unit}")
plt.ylabel("Completeness");

This gives us a sense of how the completeness behaves as a function of integration time, but only for two data points.  We can also look in more detail for a particular target.  Let's pick a somewhat bright star beyond 5 parsecs:

In [ ]:
TL = sim6.TargetList
sInd = np.where((TL.Vmag <= 7) & (TL.dist >= 5 * u.pc))[0][0]
print(f"We'll be focusing on {TL.Name[sInd]}.")
print(f"This target has a V mag of {TL.Vmag[sInd]} and is {TL.dist[sInd]} from Earth.")
print(
    f"This target has a saturation dMag of {TL.saturation_dMag[sInd] :.2f}, "
    f"and requires {TL.int_tmin[sInd].to(u.h) :.2f} of integration to get to a dMag "
    f"of {TL.int_dMag[sInd]}"
)

Let's take a look at how instrumental constraints interact with the planet population for this target:

In [ ]:
TL = sim6.TargetList # grab the TargetList
# grab the first (and only) observing mode:
mode = sim6.OpticalSystem.observingModes[0] 
fig = plt.figure()
ax = fig.add_subplot(111)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    cs = ax.contourf(
        TL.Completeness.xnew,
        TL.Completeness.ynew,
        TL.Completeness.Cpdf,
        locator=ticker.LogLocator(),
    )
ax.set_xlabel("Separation (AU)")
ax.set_ylabel("$\Delta$mag")
cbar = fig.colorbar(cs)
cbar.ax.set_title("log$_{10}(f_{\Delta\\mathrm{mag},s})$")
projIWA = np.tan(mode["IWA"]) * TL.dist[sInd]
projOWA = np.tan(mode["OWA"]) * TL.dist[sInd]
ax.plot([projIWA.to(u.AU).value] * 2, [10, 50], "--", label="IWA")
ax.plot([projOWA.to(u.AU).value] * 2, [10, 50], "--", label="OWA")
ax.plot(
    [projIWA.to(u.AU).value, projOWA.to(u.AU).value],
    [TL.int_dMag[sInd]] * 2,
    label="Nominal integration dMag",
)
ax.plot(
    [projIWA.to(u.AU).value, projOWA.to(u.AU).value],
    [TL.saturation_dMag[sInd]] * 2,
    label="Saturation integration dMag",
)
plt.legend()
ax.set_xlim([0, 30])
ax.set_ylim([10, 50]);

The heat map represents the 2D joint probability density function of this population's projected separation and delta magnitude.  The lines represent instrumental limits. Note that the $\Delta$mag curves are straight lines because the default starlight-suppression system values use the same contrast and throughput values for all separations between the inner and outer working angles.

We'll now compute integration times for a range of $\Delta$mag values and also confirm that the saturation $\Delta$mag is correct:

In [ ]:
TL = sim6.TargetList
# define an array of target delta mag values
# between just below the PDF at the IWA out to just above the
# pre-computed saturation deltaMag
dMags1 = np.linspace(17, TL.saturation_dMag[sInd] + 0.1, 100)
sInds = np.array([sInd] * len(dMags1))

# use the default values of local and exo-zodiacal light:
fZ = (
    [TL.ZodiacalLight.fZ0.value] * len(dMags1) * TL.ZodiacalLight.fZ0.unit
)  # local zodiacal light
fEZ = (
    [TL.ZodiacalLight.fEZ0.value] * len(dMags1) * TL.ZodiacalLight.fEZ0.unit
)  # exo-zodiacal light

# Use the first available observing mode
mode = TL.OpticalSystem.observingModes[0]

# use coronagraph parameters at the nominal angular separation:
WAs = [TL.int_WA[sInd].value] * len(sInds) * TL.int_WA.unit

# compute integration time
intTimes1 = TL.OpticalSystem.calc_intTime(TL, sInds, fZ, fEZ, dMags1, WAs, mode)

# plot results
plt.figure()
plt.semilogy(dMags1, intTimes1)
plt.xlabel("$\\Delta$mag")
plt.ylabel(f"Integration Time {intTimes1.unit}")
print(
    "Note that all integration time values above the "
    "saturation dMag are NaN (infeasible): "
    f"{intTimes1[dMags1 > TL.saturation_dMag[sInd]]}"
)

We can now compute completeness as a function of integration time:

In [ ]:
# first we'll remove the NaN integration times
TL = sim6.TargetList
intTimes1 = intTimes1[dMags1 < TL.saturation_dMag[sInd]]
dMags1 = dMags1[dMags1 < TL.saturation_dMag[sInd]]

# compute the projected inner and outer workign angles:
projIWA = np.tan(mode["IWA"]) * TL.dist[sInd]  # projected IWA
projOWA = np.tan(mode["OWA"]) * TL.dist[sInd]  # projected OWA

# compute completeness
comp1 = TL.Completeness.comp_calc(
    projIWA.to(u.AU).value, projOWA.to(u.AU).value, dMags1
)

plt.figure()
plt.semilogx(intTimes1, comp1)
plt.xlabel(f"Integration Time [{intTimes1.unit}]")
plt.ylabel("Completeness");

We can also invert the integration time calculation to compute the achievable $\Delta$mag as a function of integration time:

In [ ]:
TL = sim6.TargetList
sInds = np.array([sInd] * len(dMags1))
# local zodiacal light:
fZ = [TL.ZodiacalLight.fZ0.value] * len(dMags1) * TL.ZodiacalLight.fZ0.unit
# exo-zodiacal light
fEZ = [TL.ZodiacalLight.fEZ0.value] * len(dMags1) * TL.ZodiacalLight.fEZ0.unit
# use coronagraph parameters at this nominal separation
WAs = [TL.int_WA[sInd].value] * len(sInds) * TL.int_WA.unit
# compute completeness as function of integration time:
dMags2 = TL.OpticalSystem.calc_dMag_per_intTime(
    intTimes1, TL, sInds, fZ, fEZ, WAs, mode
)
print(f"Maximum difference: {np.max(np.abs(dMags1 - dMags2))}")

Finally, we have the ability to evaluate the rate of change of completeness as a function of integration time (formally, the derivative $\frac{\mathrm{d}c}{\mathrm{d}t}$):

In [ ]:
TL = sim6.TargetList
dcdt1 = TL.Completeness.dcomp_dt(
    intTimes1,
    TL,
    sInds,
    TL.ZodiacalLight.fZ0,
    TL.ZodiacalLight.fEZ0,
    TL.int_WA[sInd],
    mode,
).to("1/d")

plt.figure()
plt.semilogx(intTimes1, dcdt1)
plt.xlabel(f"Integration Time [{intTimes1.unit}]")
plt.ylabel("$\\frac{\\mathrm{d}c}{\\mathrm{d}t}$");

# 5. Observatory and Keepout

Let's set up an observatory and define our keepout regions.  We'll use a nominal Sun-Earth L$_2$ point halo orbit.  The keepout is defined for each starlight suppression system by a series of keywords defining the minimum and maximum angular separation between the line of sight and the Sun, Earth, Moon, and all other major solar system bodies.  We'll set the solar keepout as outside of 40$^\circ$ and 90$^\circ$, and ignore the others.  This is consistent with a well-baffled telescope and either solar panel-induced restrictions (from solar panels mounted orthogonally to the telescope aperture) or the keepout imposed by reflection from a starshade. 

In [ ]:
specs7 = {
    "skipSaturationCalcs": False,
    "filterBinaries": False,
    "ppFact": 0.1,
    "eta": 3,
    "arange": [0.09084645, 30],
    "smaknee": 10,
    "FoV": 2.0,
    "koAngles_Sun": [40, 90], # set solar keepout angles
    "missionLife": 1,  # set mission duration to 1 year
    "missionPortion": 1,  # allocate 100% of available time to exoplanet imaging
    "modules": {
        "PlanetPopulation": "SAG13",
        "StarCatalog": "HWOMissionStars",
        "OpticalSystem": " ",
        "ZodiacalLight": " ",
        "BackgroundSources": " ",
        "PlanetPhysicalModel": " ",
        "Observatory": "ObservatoryL2Halo",
        "TimeKeeping": " ",
        "PostProcessing": " ",
        "Completeness": "BrownCompleteness",
        "TargetList": " ",
        "SimulatedUniverse": "SAG13Universe",
        "SurveySimulation": " ",
        "SurveyEnsemble": " ",
    }
}

sim7 = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs7))

The `SurveySimulation` object automatically constructs keepout maps---boolean arrays of target availability as a function of mission time---for each defined starlight suppression system. Let's take a look at one:

In [ ]:
coords = (
    sim7.TargetList.coords.heliocentrictrueecliptic
)  # grab target coordinates in ecliptic coords
sInds = np.argsort(coords.lon)
cmap = matplotlib.colors.ListedColormap(["black", "green"])
plt.figure()
plt.pcolor(
    sim7.SurveySimulation.koTimes.value,
    np.arange(sim7.TargetList.nStars),
    sim7.SurveySimulation.koMaps[
        sim7.OpticalSystem.starlightSuppressionSystems[0]["name"]
    ][sInds],
    cmap=cmap,
)
plt.ylabel("Target Number")
plt.xlabel(f"Time [{sim7.SurveySimulation.koTimes.format}]")
cbar = plt.colorbar(ticks=[0.25, 0.75], drawedges=True)
cbar.ax.set_yticklabels(["Unavailable", "Available"]);

As we sorted the targets by their ecliptic longitudes, there is some structure in the keepout map. The regions of target availability move along with target ecliptic longitude over the course of the year.  Most targets have at least two discrete gaps in availability (corresponding to being below the minimum or above the maximum keepout angle value).  Total availability is inversely proportional to ecliptic latitude (i.e., targets near the poles are available more than targets near the ecliptic).  We can demonstrate this by summing over the rows of our keepout map (which gives us the total number of days available):  

In [ ]:
totavail = np.sum(
    sim7.SurveySimulation.koMaps[
        sim7.OpticalSystem.starlightSuppressionSystems[0]["name"]
    ],
    axis=1,
)
plt.figure()
plt.scatter(coords.lat, totavail)
plt.xlabel("Ecliptic Latitude [deg]")
plt.ylabel("Total Target Availability [days]");

While we're here, let's also take a quick look at the orbit our observatory is on:

In [ ]:
orbit = sim7.Observatory.orbit(sim7.SurveySimulation.koTimes)
ax = plt.figure().add_subplot(projection="3d")

ax.plot(orbit[:, 0], orbit[:, 1], orbit[:, 2])
ax.set_xlabel("x (AU)")
ax.set_ylabel("y (AU)")
ax.set_zlabel("z (AU)");

Unsurprisingly, this looks a lot like Earth's orbit (it's actually tracking the orbit of $L_2$, which orbits slightly beyond Earth).  In order to see the periodicity of the halo, we need to plot this in the rotating frame:

In [ ]:
rotframeorbit = sim7.Observatory.haloPosition(sim7.SurveySimulation.koTimes)
ax = plt.figure().add_subplot(projection="3d")

ax.plot(rotframeorbit[:, 0], rotframeorbit[:, 1], rotframeorbit[:, 2])
ax.set_xlabel("x (AU)")
ax.set_ylabel("y (AU)")
ax.set_zlabel("z (AU)");

# 6. Running a Simulation

Finally, let's create a new instance of our final simulation configuration and actually run the survey:

In [ ]:
specs7 = {
    "skipSaturationCalcs": False,
    "filterBinaries": False,
    "ppFact": 0.1,
    "eta": 3,
    "arange": [0.09084645, 30],
    "smaknee": 10,
    "FoV": 2.0,
    "koAngles_Sun": [40, 90], # set solar keepout angles
    "missionLife": 1,  # set mission duration to 1 year
    "missionPortion": 1,  # allocate 100% of available time to exoplanet imaging
    "modules": {
        "PlanetPopulation": "SAG13",
        "StarCatalog": "HWOMissionStars",
        "OpticalSystem": " ",
        "ZodiacalLight": " ",
        "BackgroundSources": " ",
        "PlanetPhysicalModel": " ",
        "Observatory": "ObservatoryL2Halo",
        "TimeKeeping": " ",
        "PostProcessing": " ",
        "Completeness": "BrownCompleteness",
        "TargetList": " ",
        "SimulatedUniverse": "SAG13Universe",
        "SurveySimulation": " ",
        "SurveyEnsemble": " ",
    }
}
sim7 = EXOSIMS.MissionSim.MissionSim(**copy.deepcopy(specs7))

sim7.run_sim()

The simulated set of observations is encoded in a list of dictionaries stored in attribute `sim.SurveySimulation.DRM`. Each dictionary includes information about the observation.  We can look at one of them to see what information is available:

In [ ]:
sim7.SurveySimulation.DRM[0]

We can quickly collect information about all observations via python's list comprehensions. For example, we can identify which stars we observed in this simulation:

In [ ]:
sInds = [row["star_ind"] for row in sim7.SurveySimulation.DRM]  # observed stars

ra = sim7.TargetList.coords.ra.wrap_at(180 * u.degree).rad  # all stars
dec = sim7.TargetList.coords.dec.rad

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection="mollweide")
p = ax.scatter(ra, dec, c=sim7.TargetList.int_comp)
ax.grid(True)
plt.colorbar(p, label="Completeness")
diffs = np.abs(np.diff(ra[sInds]))
for j in range(1, len(sInds)):
    if diffs[j - 1] < np.pi:
        plt.plot(ra[sInds[j - 1 : j + 1]], dec[sInds[j - 1 : j + 1]], "k");

Because this simulation did not include a starshade, there is little penalty for transitioning between targets that are further apart from one another, and so we see the scheduler prioritizing higher-completeness targets whenever they are available.

Let's see what happens if we try to run the simulation again:

In [ ]:
sim7.run_sim()

Unsurprisingly, we're out of time.  Before we can re-run the simulation, we have to reset, which we can do via the `reset_sim` method:

In [ ]:
sim7.reset_sim?

Note that we have the option to generate entirely new planets (default) or to simply rewind the planets we already have back to their initial positions.  Let's try the latter:

In [ ]:
sim7.reset_sim(genNewPlanets=False)

In [ ]:
sim7.run_sim()

We see that we ended up with the essentially the same mission schedule and outcomes as with our first attempt. 

# Exercise: Choose another star and re-create some or all of the integration time calculations

Have fun!